In [1]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('dat-test.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [4]:
data = pd.read_csv('../../lesson-15/code/data/rossmann.csv', low_memory=False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [6]:
sql.read_sql('select * from rossmann_sales limit 10', con=conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [7]:
rossmann_stores = pd.read_csv('data/rossmann-stores.csv')
rossmann_stores.to_sql('rossmann_stores', if_exists='replace', index=False, con=conn)

In [8]:
sql.read_sql(
"""
SELECT 
Store, Sales 
FROM rossmann_sales;
""", con=conn).head()

,Store,Sales
0,1,5263
1,2,6064
2,3,8314
3,4,13995
4,5,4822


> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [9]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers 
FROM rossmann_sales;
""", con=conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


In [10]:
sql.read_sql(
"""
SELECT 
Store, Sales
FROM rossmann_sales
WHERE Store = 1;
""", con=conn).head()

,Store,Sales
0,1,5263
1,1,5020
2,1,4782
3,1,5011
4,1,6102


> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [11]:
sql.read_sql(
"""
SELECT 
Store, Date, Customers
FROM rossmann_sales
WHERE Open = 1 and Promo = 1;
""", con=conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


In [12]:
sql.read_sql(
"""
SELECT
Store, SUM(Sales), AVG(Customers)
FROM rossmann_sales
WHERE Open = 1
GROUP BY Store
""", con=conn).head()

,Store,SUM(Sales),AVG(Customers)
0,1,3716854,564.049936
1,2,3883858,583.998724
2,3,5408261,750.077022
3,4,7556507,1321.752551
4,5,3642818,537.340180


> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [13]:
sql.read_sql(
"""
SELECT
Promo, Store, SUM(Sales)
FROM rossmann_sales
GROUP BY Promo
""", con=conn).head()

,Promo,Store,SUM(Sales)
0,0,1115,2771974337
1,1,1115,3101206286


In [14]:
sql.read_sql(
"""
SELECT 
Store, SUM(Sales) as total_sales, AVG(Customers)
FROM rossmann_sales
WHERE Open = 1
GROUP BY Store
ORDER BY total_sales desc
""", con=conn).head()

,Store,total_sales,AVG(Customers)
0,262,19516842,3402.010616
1,817,17057867,3130.573980
2,562,16927322,3105.053079
3,1114,16202585,3200.946429
4,251,14896870,2450.492940


In [15]:
sql.read_sql(
"""
SELECT a.Store, a.Sales, s.CompetitionDistance
FROM rossmann_sales a
JOIN rossmann_stores s 
ON a.Store = s.Store
""", con=conn).head()

,Store,Sales,CompetitionDistance
0,1,5263,1270.0
1,2,6064,570.0
2,3,8314,14130.0
3,4,13995,620.0
4,5,4822,29910.0
